In [92]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report,r2_score, mean_squared_error

In [93]:
#EDA
df=pd.read_csv("PB_All_2000_2021.csv", sep=";")
print("\n Sample of Table \n", df.head(10))
print("\n Info of Table \n")
df.info()
print("\n Shape of Table \n",df.shape)
print("\n Stastics of Table \n",df.describe())
print("\n Null characters of Table \n",df.isnull().sum())


 Sample of Table 
    id        date    NH4  BSK5  Suspended     O2     NO3    NO2    SO4    PO4  \
0   1  17.02.2000  0.330  2.77       12.0  12.30   9.500  0.057  154.0  0.454   
1   1  11.05.2000  0.044  3.00       51.6  14.61  17.750  0.034  352.0  0.090   
2   1  11.09.2000  0.032  2.10       24.5   9.87  13.800  0.173  416.0  0.200   
3   1  13.12.2000  0.170  2.23       35.6  12.40  17.130  0.099  275.2  0.377   
4   1  02.03.2001  0.000  3.03       48.8  14.69  10.000  0.065  281.6  0.134   
5   1  07.06.2001  0.020  4.02       34.0  10.61  11.800  0.016  287.0  0.208   
6   1  10.09.2001  0.863  3.91      147.0  10.96  20.500  0.284  595.2  0.674   
7   1  06.11.2001  0.060  2.97       71.2  13.47  25.800  0.095  314.0  0.390   
8   1  12.03.2002  0.168  4.15       27.0  17.82   3.945  0.058  153.6  0.110   
9   1  06.06.2002  0.001  7.11       74.4  19.28   2.260  0.017  409.6  0.181   

       CL  
0   289.5  
1  1792.0  
2  2509.0  
3  1264.0  
4  1462.0  
5  1183.0  
6  4

In [94]:
#Changing date format
df['date']=pd.to_datetime(df['date'], format='%d.%m.%Y')
df=df.sort_values(by=['id','date'])
df.head(10)

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,2000-02-17,0.330,2.77,12.0,12.30,9.500,0.057,154.0,0.454,289.5
1,1,2000-05-11,0.044,3.00,51.6,14.61,17.750,0.034,352.0,0.090,1792.0
2,1,2000-09-11,0.032,2.10,24.5,9.87,13.800,0.173,416.0,0.200,2509.0
3,1,2000-12-13,0.170,2.23,35.6,12.40,17.130,0.099,275.2,0.377,1264.0
4,1,2001-03-02,0.000,3.03,48.8,14.69,10.000,0.065,281.6,0.134,1462.0
5,1,2001-06-07,0.020,4.02,34.0,10.61,11.800,0.016,287.0,0.208,1183.0
6,1,2001-09-10,0.863,3.91,147.0,10.96,20.500,0.284,595.2,0.674,4023.0
7,1,2001-11-06,0.060,2.97,71.2,13.47,25.800,0.095,314.0,0.390,1907.0
8,1,2002-03-12,0.168,4.15,27.0,17.82,3.945,0.058,153.6,0.110,473.0
9,1,2002-06-06,0.001,7.11,74.4,19.28,2.260,0.017,409.6,0.181,1782.0


In [104]:
# Removing date collumn and adding year, month and day
df['year']= df['date'].dt.year
df['month']= df['date'].dt.month
df['day'] = df['date'].dt.day
df=df.sort_values(by=['id','year','month','day'])
df.drop('date', axis=1, inplace=True)
df.head(10)

,id,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL,year,month,day
0,1,0.330,2.77,12.0,12.30,9.500,0.057,154.0,0.454,289.5,2000,2,17
1,1,0.044,3.00,51.6,14.61,17.750,0.034,352.0,0.090,1792.0,2000,5,11
2,1,0.032,2.10,24.5,9.87,13.800,0.173,416.0,0.200,2509.0,2000,9,11
3,1,0.170,2.23,35.6,12.40,17.130,0.099,275.2,0.377,1264.0,2000,12,13
4,1,0.000,3.03,48.8,14.69,10.000,0.065,281.6,0.134,1462.0,2001,3,2
5,1,0.020,4.02,34.0,10.61,11.800,0.016,287.0,0.208,1183.0,2001,6,7
6,1,0.863,3.91,147.0,10.96,20.500,0.284,595.2,0.674,4023.0,2001,9,10
7,1,0.060,2.97,71.2,13.47,25.800,0.095,314.0,0.390,1907.0,2001,11,6
8,1,0.168,4.15,27.0,17.82,3.945,0.058,153.6,0.110,473.0,2002,3,12
9,1,0.001,7.11,74.4,19.28,2.260,0.017,409.6,0.181,1782.0,2002,6,6


In [112]:
# Reorder the DataFrame
cols = df.columns.tolist()
cols.insert(cols.index('id') + 1, cols.pop(cols.index('year')))
cols.insert(cols.index('id') + 2, cols.pop(cols.index('month')))
cols.insert(cols.index('id') + 3, cols.pop(cols.index('day')))
df = df[cols]
df.head()

,id,year,month,day,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,2000,2,17,0.330,2.77,12.0,12.30,9.50,0.057,154.0,0.454,289.5
1,1,2000,5,11,0.044,3.00,51.6,14.61,17.75,0.034,352.0,0.090,1792.0
2,1,2000,9,11,0.032,2.10,24.5,9.87,13.80,0.173,416.0,0.200,2509.0
3,1,2000,12,13,0.170,2.23,35.6,12.40,17.13,0.099,275.2,0.377,1264.0
4,1,2001,3,2,0.000,3.03,48.8,14.69,10.00,0.065,281.6,0.134,1462.0


In [ ]:
df.columns
componentsof_water=['NH4', 'BSK5', 'Suspended', 'O2', 'NO3', 'NO2', 'SO4','PO4', 'CL']